# A full training

Install the Datasets, Evaluate, Transformers and Accelerate libraries to run this notebook.

In [1]:
!pip install -q datasets evaluate transformers[sentencepiece]
!pip install -q accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.2 MB/s eta 0:00:00


To run the training on a TPU, you will need to uncomment and run the following line:

In [2]:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

We shall load the "`mrpc`" subset (a.k.a. 'configuration') of the `"glue"` dataset.

From the <a href="https://huggingface.co/datasets/glue" target="_blank">dataset page</a>:

> GLUE, the General Language Understanding Evaluation benchmark (https://gluebenchmark.com/) is a collection of resources for training, evaluating, and analyzing natural language understanding systems.

`"mrpc"`:

> The Microsoft Research Paraphrase Corpus (Dolan & Brockett, 2005) is a corpus of sentence pairs automatically extracted from online news sources, with human annotations for whether the sentences in the pair are semantically equivalent.

From the <a href="https://huggingface.co/docs/datasets/v2.16.1/en/package_reference/loading_methods#datasets.load_dataset" target="_blank">documentation</a> of `load_dataset`, the first two parameters are:

- **path** (`str`) — Path or name of the dataset.

- **name** (`str`, *optional*) — Defining the name of the dataset configuration.

In [4]:
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [5]:
features = raw_datasets['train'].features
features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [6]:
id2label = {id: label for id, label in enumerate(features['label'].names)}
id2label

{0: 'not_equivalent', 1: 'equivalent'}

In [7]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.model_max_length

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

512

In [8]:
def tokenize_function(example):
    return tokenizer(example['sentence1'], example['sentence2'], truncation=True, max_length=512)

In [9]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

**Note:** The tokenization was pretty straight forward (compared, for example to the tokenization approach in The Kaggle Book.)

In [10]:
example0 = tokenized_datasets['train'][0]
type(example0)

dict

In [11]:
for k, v in example0.items():
    print(f"{k}: {v}")

sentence1: Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .
sentence2: Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .
label: 1
idx: 0
input_ids: [101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102]
token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [12]:
id2label[example0['label']]

'equivalent'

In [13]:
len(example0['input_ids']), len(example0['token_type_ids']), len(example0['attention_mask'])

(50, 50, 50)

In [14]:
example1 = tokenized_datasets['train'][1]
type(example1)

dict

In [15]:
for k, v in example1.items():
    print(f"{k}: {v}")

sentence1: Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .
sentence2: Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .
label: 0
idx: 1
input_ids: [101, 9805, 3540, 11514, 2050, 3079, 11282, 2243, 1005, 1055, 2077, 4855, 1996, 4677, 2000, 3647, 4576, 1999, 2687, 2005, 1002, 1016, 1012, 1019, 4551, 1012, 102, 9805, 3540, 11514, 2050, 4149, 11282, 2243, 1005, 1055, 1999, 2786, 2005, 1002, 6353, 2509, 2454, 1998, 2853, 2009, 2000, 3647, 4576, 2005, 1002, 1015, 1012, 1022, 4551, 1999, 2687, 1012, 102]
token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [16]:
id2label[example1['label']]

'not_equivalent'

In [17]:
len(example1['input_ids']), len(example1['token_type_ids']), len(example1['attention_mask'])

(59, 59, 59)

`DataCollatorWithPadding` will allow us to use dynamic padding for our batches.

In [18]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [19]:
tokenized_datasets = tokenized_datasets.remove_columns(['sentence1', 'sentence2', 'idx'])
tokenized_datasets = tokenized_datasets.rename_column('label', 'labels')
tokenized_datasets.set_format("torch")
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

**Note:**

1. When using the Trainer API, we don't have to rename `'label'` to `'labels'`. The Trainer API takes care of it.
2. `set_format` is the inplace version of `with_format`.

In [20]:
example0 = tokenized_datasets['train'][0]
type(example0)

dict

In [21]:
for k, v in example0.items():
    print(f"{k}: {v}")

labels: 1
input_ids: tensor([  101,  2572,  3217,  5831,  5496,  2010,  2567,  1010,  3183,  2002,
         2170,  1000,  1996,  7409,  1000,  1010,  1997,  9969,  4487, 23809,
         3436,  2010,  3350,  1012,   102,  7727,  2000,  2032,  2004,  2069,
         1000,  1996,  7409,  1000,  1010,  2572,  3217,  5831,  5496,  2010,
         2567,  1997,  9969,  4487, 23809,  3436,  2010,  3350,  1012,   102])
token_type_ids: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1])
attention_mask: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1])


In [22]:
example0['labels']

tensor(1)

In [23]:
id2label[example0['labels'].item()]

'equivalent'

In [24]:
example0['labels'].shape, example0['input_ids'].shape, example0['token_type_ids'].shape, example0['attention_mask'].shape

(torch.Size([]), torch.Size([50]), torch.Size([50]), torch.Size([50]))

In [25]:
example1 = tokenized_datasets['train'][1]
type(example1)

dict

In [26]:
for k, v in example1.items():
    print(f"{k}: {v}")

labels: 0
input_ids: tensor([  101,  9805,  3540, 11514,  2050,  3079, 11282,  2243,  1005,  1055,
         2077,  4855,  1996,  4677,  2000,  3647,  4576,  1999,  2687,  2005,
         1002,  1016,  1012,  1019,  4551,  1012,   102,  9805,  3540, 11514,
         2050,  4149, 11282,  2243,  1005,  1055,  1999,  2786,  2005,  1002,
         6353,  2509,  2454,  1998,  2853,  2009,  2000,  3647,  4576,  2005,
         1002,  1015,  1012,  1022,  4551,  1999,  2687,  1012,   102])
token_type_ids: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
attention_mask: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [27]:
example1['labels']

tensor(0)

In [28]:
id2label[example1['labels'].item()]

'not_equivalent'

In [29]:
example1['labels'].shape, example1['input_ids'].shape, example1['token_type_ids'].shape, example1['attention_mask'].shape

(torch.Size([]), torch.Size([59]), torch.Size([59]), torch.Size([59]))

In [30]:
tokenized_datasets['train'].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [31]:
batch_size = 8

In [32]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets['train'],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets['validation'],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator
)

len(train_dataloader), len(eval_dataloader)

(459, 51)

There are 459 batches in `train_dataloader` and 51 batches in `eval_dataloader`.

In [33]:
for batch in train_dataloader:
    break
type(batch)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


transformers.tokenization_utils_base.BatchEncoding

**Note:** A batch is a `BatchEncoding` object, not a `dict` (even though it looks and works similar to a `dict`).

In [34]:
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 81]),
 'token_type_ids': torch.Size([8, 81]),
 'attention_mask': torch.Size([8, 81])}

In [35]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [36]:
outputs = model(**batch)
outputs

SequenceClassifierOutput(loss=tensor(0.5713, grad_fn=<NllLossBackward0>), logits=tensor([[-1.0599,  0.3698],
        [-1.0569,  0.3868],
        [-1.0674,  0.3722],
        [-1.0605,  0.3777],
        [-1.0434,  0.3737],
        [-1.0600,  0.3886],
        [-1.0438,  0.3642],
        [-1.0578,  0.3707]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

**Note:** `loss` is an attribute of `SequenceClassifierOutput`. If `'labels'` isn't part of the batch, then `loss` will be `None`. This is shown below.

In [37]:
model(**{'input_ids': batch['input_ids'], 'token_type_ids': batch['token_type_ids'], 'attention_mask': batch['attention_mask']})

SequenceClassifierOutput(loss=None, logits=tensor([[-1.0599,  0.3698],
        [-1.0569,  0.3868],
        [-1.0674,  0.3722],
        [-1.0605,  0.3777],
        [-1.0434,  0.3737],
        [-1.0600,  0.3886],
        [-1.0438,  0.3642],
        [-1.0578,  0.3707]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

It should be possible to override the default loss function with a different / custom loss function by sub-classing `BertForSequenceClassification`. You can refer to Chapter 4 of the Hugging Face book for this.

In [38]:
print(outputs.loss, outputs.logits.shape)

tensor(0.5713, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [39]:
learning_rate = 5e-5

**Note:** The learning rate is significantly smaller than `3e-4` (which works well for ResNet34).

In [40]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=learning_rate)

**Note:** When using linear decay, the value of `lr` specified above will actually be the maximum value. The minimum value will be `0`.

In [41]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
num_training_steps

1377

In [42]:
scheduler_type = "linear"
num_warmup_steps = 0
lr_scheduler = get_scheduler(
    scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)
lr_scheduler

**Note:** `get_scheduler` is a convenience function provided by the Transformers library. But it can be when training pure PyTorch models as well!

In [43]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else torch.device("cpu"))
device

device(type='cuda')

In [44]:
model = model.to(device)

Are the parameters of the model's body frozen? Let's check the first layer to find out.

In [45]:
for name, p in model.named_parameters():
    print(type(name))
    print(name)
    print(type(p))
    print(p.shape)
    print(p.requires_grad)
    break

<class 'str'>
bert.embeddings.word_embeddings.weight
<class 'torch.nn.parameter.Parameter'>
torch.Size([30522, 768])
True


No! That means that we're fine-tuning all the layers, not just the randomly instantiated head. (We're also not using discriminative learning rates.)

In [46]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step() # Note: Here, we're updating the learning rate after each step (not after each epoch).
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

In [47]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch['labels'])

metric.compute()

{'accuracy': 0.8578431372549019, 'f1': 0.9003436426116839}

---

In [ ]:
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

accelerator = Accelerator()

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
from accelerate import notebook_launcher

notebook_launcher(training_function)